## Call local ERP System using local LLM

### Requisites

Install required modules if not already done

In [1]:
# !pip install pyautogen

For reference, I added the variable. If True, it will call the localhost, if False, it will call OpenAI's model.

In [2]:
use_local = False

Import required modules and define the configurations for the models.
The local files should look eiher

for OpenAI
[
    {
        "model": "gpt-3.5-turbo",
        "api_key": "sk-..."
    }
]

or for localhost

[
    {
        "model": "mistral-7b-instruct-v0.2.Q4_K_M",
        "api_key": "sk-11111111111111111111111111111111",
        "base_url": "http://localhost:8000/v1"
    }
]

In [10]:
import autogen

file = "OAI_CONFIG_LIST.json"
model = "gpt-3.5-turbo"
if use_local:
    file = "OAI_CONFIG_LIST_LOCAL.json"
    model = "mistral-7b-instruct-v0.2.Q4_K_M"

config_list = autogen.config_list_from_json(
    file,
    filter_dict={
        "model": [model]
    }
)

llm_config = {
    "timeout": 600,
    "seed": 42,
    "config_list": config_list,
    "temperature": 0.5,
    "functions": [
        {
            "name": "get_description_of_workcenter",
            "description": "Get the ERP description of a given workcenter id",
            "parameters": {
                "type": "object",
                "properties": {
                    "workcenter_id": {
                        "type": "string",
                        "description": "The ERP workcenter id"
                    }
                },
                "required": ["workcenter_id"]
            }
        },
        {
            "name": "get_current_orderdata_for_workcenter",
            "description": "Get the ERP order for a given workcenter",
            "parameters": {
                "type": "object",
                "properties": {
                    "workcenter_id": {
                        "type": "string",
                        "description": "The ERP workcenter id"
                    }
                },
                "required": ["workcenter_id"]
            }
        }
    ]
}

Define the functions (I first tried to have them in a separate file and import them, that didn't work either, only with OpenAI's API)

In [4]:
from skills import *
print(get_description_of_workcenter('511V0001'))
print(get_current_orderdata_for_workcenter('511V0001'))

Verpackungsanlage 441
{'machineId': 'M1002976', 'localName': 'Montagelinie A55', 'workcenter': '511V0B01', 'location': 'Factory A1', 'doesSendToERP': False, 'hasCounterData': False, 'material': '3232975117A', 'order': '1234567', 'confirmation': '1827398172'}


Define agents

### User Proxy
Simulation of user, can execute code

### Assistant


In [5]:
assistant_system_message = "You're a helpful asssistant."

if use_local: 
    assistant_system_message = """PROVIDE USEFUL PYTHON CODE INCORPORATING THE LOCALLY AVAILABLE FUNCTIONS FOR ERP REQUESTS. You are being given the following python functions with description. Assume the functions available to the user, without having to import anything
    function name: get_description_of_workcenter
    argument (string, use quotes): workcenter_id
    description: Get the description of a given workcenter id

    function name: get_current_orderdata_for_workcenter
    argument (string, use quotes): workcenter_id
    description: Get the order for a given workcenter

    Use those functions when answering. 
    """

assistant = autogen.AssistantAgent(
    name='assistant',
    llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "") and "terminate" in x.get("content", "").lower(),
    system_message=assistant_system_message,
)

user_proxy = autogen.UserProxyAgent(
    name='user_proxy',
    human_input_mode='NEVER',
    max_consecutive_auto_reply=4,
    is_termination_msg=lambda x: x.get("content", "") and "terminate" in x.get("content", "").lower(),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False
    },
    llm_config=llm_config,
    system_message="""You are a local ERP API assistant, you know the functions get_description_of_workcenter and get_current_orderdata_for_workcenter.
    If the task was answered to full satisfaction, summarize with repeating the task and add the result and end the answer with the word TERMINATE.""",
)

user_proxy.register_function(
    function_map={
        "get_description_of_workcenter": get_description_of_workcenter,
        "get_current_orderdata_for_workcenter": get_current_orderdata_for_workcenter
    }
)
assistant.register_function(
    function_map={
        "get_description_of_workcenter": get_description_of_workcenter,
        "get_current_orderdata_for_workcenter": get_current_orderdata_for_workcenter
    }
)




Since it doesn't work with import skills, try to define it here

In [6]:
if False:
    @user_proxy.register_for_execution()
    @assistant.register_for_llm(description='Userfull for getting the name of the workcenter')
    def get_description_of_workcenter(workcenter_id: str) -> str:
        """
        Calls the ERP api and requests the description of a ERP Workcenter Id. This is a helper function.

        Args:
            query (str): The ERP workcenter id.

        Returns:
            string: The description of the ERP workcenter id

        Example:
            >>> results = get_description_of_workcenter("10110101")
            >>> print(results) # prints e.g. "Waschanlage 1234"
        """
        return random.choice(["Stanzanlage 3.12", "Montage M1233", "Verpackungsanlage 441", "Montagelinie A55"])


In [8]:

user_proxy.initiate_chat(
    assistant,
    message="What name does workcenter 511V0001 have? "
)

user_proxy (to assistant):

What name does workcenter 511V0001 have? 

--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested function Call: get_description_of_workcenter *****
Arguments: 
{
  "workcenter_id": "511V0001"
}
******************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_description_of_workcenter...
user_proxy (to assistant):

***** Response from calling function "get_description_of_workcenter" *****
Verpackungsanlage 441
**************************************************************************

--------------------------------------------------------------------------------
assistant (to user_proxy):

The workcenter 511V0001 is named "Verpackungsanlage 441".

--------------------------------------------------------------------------------
user_proxy (to assistant):

The workcenter 511